<a href="https://colab.research.google.com/github/Seyoung-Jung/DOTORY/blob/main/kogpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라벨링 데이터 적용

- kogpt2 사용
- 테마 & 등장인물 입력 추가

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Packages

In [ ]:
!pip install transformers
!pip install torch
!pip install git+https://github.com/ssut/py-hanspell.git
!pip3 install pyjosa

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 50.1 MB/s 
     |████████████████████████████████| 6.5 MB 46.2 MB/s 
     |████████████████████████████████| 596 kB 61.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-6txfizfz
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-6txfizfz
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=d97631382e9a46731d0b505f36967f48cef9e66f50ecded52a7b12ed98840186
  Stored in directory: /tmp/pip-ephem-wheel-cache-irnzky34/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
import transformers
transformers.__version__

'4.17.0'

In [ ]:
import torch
torch.__version__

'1.10.0+cu111'

## Train

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
print('Length of tokenizer before add special tokens:', len(tokenizer))
tokenizer.add_special_tokens({
    'additional_special_tokens':['<우주>', '<숲속>', '<바다>', '<마을>', '<왕국>', '<기타>',
                                 '<등장인물1>', '<등장인물2>']
})
print('Length of tokenizer after add special tokens:', len(tokenizer))

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.resize_token_embeddings(len(tokenizer))
model.to(device)
model.train()

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Length of tokenizer before add special tokens: 51200
Length of tokenizer after add special tokens: 51208


Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51208, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

transformer 고유 Trainer 사용 코드

In [ ]:
from transformers import DataCollatorForLanguageModeling, LineByLineTextDataset

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # for batch
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path='/content/drive/MyDrive/DOTORY/labeled_5sentence_676.txt', block_size=128)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import TrainingArguments, Trainer

EPOCHS = 20

train_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/DOTORY/labeled_5sentence_676_checkpoint',
    overwrite_output_dir = True,
    dataloader_drop_last = True,
    per_device_train_batch_size = 32,
    learning_rate = 5e-5,
    num_train_epochs = EPOCHS,
    save_steps = 2000
)
trainer = Trainer(
    model = model,
    args = train_args,
    data_collator = data_collator,
    train_dataset = dataset
)
trainer.train()
trainer.save_model()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Step,Training Loss
500,2.795700
1000,1.812100
1500,1.263700
2000,0.943100
2500,0.778300


Saving model checkpoint to /content/drive/MyDrive/DOTORY/labeled_5sentence_676_checkpoint/checkpoint-2000
Configuration saved in /content/drive/MyDrive/DOTORY/labeled_5sentence_676_checkpoint/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/DOTORY/labeled_5sentence_676_checkpoint/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/DOTORY/labeled_5sentence_676_checkpoint
Configuration saved in /content/drive/MyDrive/DOTORY/labeled_5sentence_676_checkpoint/config.json
Model weights saved in /content/drive/MyDrive/DOTORY/labeled_5sentence_676_checkpoint/pytorch_model.bin


pytorch handwritten 코드

In [ ]:
from transformers import LineByLineTextDataset
from torch.utils.data import DataLoader
import torch

dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path='/content/drive/MyDrive/DOTORY/labeled_5sentence_676.txt', block_size=128)
dataloader = DataLoader(dataset, pin_memory=True) # shuffle=True, batch_size=BATCH_SIZE, drop_last=True

learning_rate = 5e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
EPOCH = 10
count = 0

for epoch in range(EPOCH):

    for data in dataloader:
        optimizer.zero_grad()
        data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.

        data= data.transpose(1,0)
        data= data.to(device)
        
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss.backward()
        optimizer.step()
        
        if (count > 0 and count % 100 == 0):  # or (len(data) < BATCH_SIZE)
            print('Epoch no.{} train no.{}  loss = {}' . format(epoch, count, loss))
            torch.save({
                'epoch': epoch,
                'train_no': count,
                'loss':loss,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, '/content/drive/MyDrive/DOTORY/5sentence_gender+color_checkpoint.tar')

        count += 1

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Creating features from dataset file at /content/drive/MyDrive/DOTORY/5sentence_df_gender+color.txt


TypeError: ignored

## Generate

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import nltk
from nltk.tokenize import sent_tokenize
from hanspell import spell_checker
from pyjosa.josa import Josa
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
tokenizer.add_special_tokens({
    'additional_special_tokens':['<우주>', '<숲속>', '<바다>', '<마을>', '<왕국>', '<기타>',
                                 '<등장인물1>', '<등장인물2>']
})
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/DOTORY/kogpt2_new ver/labeled_5sentence_344_checkpoint')
model.to(device)

nltk.download('punkt')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

필터링 및  첫 문장 데이터 로드

In [ ]:
f = open("/content/drive/MyDrive/DOTORY/filtering.txt", 'r')
filtering = f.read().split('\n')
f.close()

In [ ]:
import pandas as pd

f = open("/content/drive/MyDrive/DOTORY/labeled_first_sentence_344.txt", 'r')
first_sentence = f.read().split('\n')
f.close()
first_sentence.remove('')

first_sentence = pd.Series(first_sentence)
first_sentence_df = pd.concat([first_sentence.str[:4], first_sentence.str[5:]], axis=1, keys=['theme', 'first_sentence'])

In [ ]:
# def replace_name(sentence, character1, character2):
#     if '> ' in sentence:
#         sentence = sentence.replace('> ', '>')

#     if '<등장인물1>' in sentence or '<등장인물 1>' in sentence:
#         sentence = sentence.replace('<등장인물1>', character1)
#         sentence = sentence.replace('<등장인물 1>', character1)
#         if sentence[sentence.find(character1)+len(character1)] in ['을', '를', '은', '는', '이', '가', '과', '와', '이나', '나', '으로', '로', '아', '야',
#                                                                    '이랑', '랑', '이며', '며', '이다', '다', '이가', '가']:
#             idx = sentence.index(character1)+len(character1)
#             sentence = list(sentence)
#             sentence[idx] = Josa.get_josa(character1, sentence[idx])
#             sentence = "".join(sentence)

#     if '<등장인물2>' in sentence or '<등장인물 2>' in sentence:
#         sentence = sentence.replace('<등장인물2>', character2)
#         sentence = sentence.replace('<등장인물 2>', character2)
#         if sentence[sentence.find(character2)+len(character2)] in ['을', '를', '은', '는', '이', '가', '과', '와', '이나', '나', '으로', '로', '아', '야',
#                                                                    '이랑', '랑', '이며', '며', '이다', '다', '이가', '가']:
#             idx = sentence.index(character2)+len(character2)
#             sentence = list(sentence)
#             sentence[idx] = Josa.get_josa(character2, sentence[idx])
#             sentence = "".join(sentence)
#     return sentence

In [ ]:
def replace_name(sentence, character1, character2):
    if '> ' in sentence:
        sentence = sentence.replace('> ', '>')

    if '<등장인물1>' in sentence or '<등장인물 1>' in sentence:
        sentence = sentence.replace('<등장인물1>', character1)
        sentence = sentence.replace('<등장인물 1>', character1)
        idx = sentence.index(character1)+len(character1)
        if sentence[idx] in ['을', '를', '은', '는', '이', '가', '과', '와', '이나', '나', '으로', '로', '아', '야',
                                                                   '이랑', '랑', '이며', '며', '이다', '다', '이가', '가']:
            sentence = list(sentence)
            sentence[idx] = Josa.get_josa(character1, sentence[idx])
            sentence = "".join(sentence)

    if '<등장인물2>' in sentence or '<등장인물 2>' in sentence:
        sentence = sentence.replace('<등장인물2>', character2)
        sentence = sentence.replace('<등장인물 2>', character2)
        idx = sentence.index(character2)+len(character2)
        if sentence[idx] in ['을', '를', '은', '는', '이', '가', '과', '와', '이나', '나', '으로', '로', '아', '야',
                                                                   '이랑', '랑', '이며', '며', '이다', '다', '이가', '가']:
            sentence = list(sentence)
            sentence[idx] = Josa.get_josa(character2, sentence[idx])
            sentence = "".join(sentence)
    return sentence

In [ ]:
theme = input("Choose the theme between <우주>, <숲속>, <바다>, <마을>, <왕국>, <기타> : ")
character1 = input("Enter the name of character1 : ")
character2 = input("Enter the name of character2 : ")

first_sentence_theme = first_sentence_df[first_sentence_df.theme == theme].first_sentence.values

Choose the theme between <우주>, <숲속>, <바다>, <마을>, <왕국>, <기타> : <숲속>
Enter the name of character1 : 토끼
Enter the name of character2 : 호랑이


In [ ]:
result = []

input_sentence = random.choice(first_sentence_theme)
input_sentence = replace_name(input_sentence, character1, character2)
print('First sentence : ', input_sentence)
result.append(input_sentence)
encoded = torch.LongTensor().to(device)
# sentence_num = 5
# shift_num = 0

for i in range(20):
    # if i+1 > sentence_num:shift_num = len(tokenizer.encode(input_sentence))
    encoded = torch.cat([encoded, torch.tensor(tokenizer.encode(input_sentence)).to(device)])  # [shift_num:]
    generated = model.generate(encoded.unsqueeze(0), do_sample=True, use_cache=True, top_p=0.9, \
                               num_return_sequences=3, max_length=len(encoded)+100, min_length=len(encoded), \
                               temperature=0.6, pad_token_id=tokenizer.eos_token_id).to(device)  # length_penalty=10,
    generated = [generated[i][len(encoded):] for i in range(3)]  # input 중복 제거
    decoded = [tokenizer.decode(generated[i], clean_up_tokenization_spaces=True) for i in range(3)]  # decode
#     output = [decoded[i][:re.compile('[\.?!]').search(decoded[i]).start()+1] \
#               if decoded[i] else decoded[i] for i in range(3)]  # 첫번째 문장 분리
    output = [sent_tokenize(decoded[i])[0] if decoded[i] else decoded[i] for i in range(3)]  # 첫번째 문장 분리
    output = [replace_name(output[i], character1, character2) for i in range(3)]  # 등장인물 스위칭
    output = [spell_checker.check(output[i]).checked for i in range(3)]  # 맞춤법 검사
    for i in range(3):
        if sum([f in output[i] for f in filtering]):  # filter it and new generate
            generated = model.generate(encoded.unsqueeze(0), do_sample=True, use_cache=True, top_p=0.9, \
                                num_return_sequences=1, max_length=len(encoded)+100, min_length=len(encoded), \
                                temperature=0.6, pad_token_id=tokenizer.eos_token_id).to(device)
            generated = generated[0][len(encoded):]
            decoded = tokenizer.decode(generated, clean_up_tokenization_spaces=True)
#             try:output[i] = decoded[:re.compile('[\.?!]').search(decoded[i]).start()+1]
            try:output[i] = sent_tokenize(decoded)
            except:output[i] = decoded
            output[i] = replace_name(output[i], character1, character2)
            output[i] = spell_checker.check(output[i]).checked
        print('Generated ' + str(i+1) + ' :', output[i])
    next_idx = int(input('\nChoose the sentence number : ')) - 1
    print('Selected sentence : ', output[next_idx])
    result.append(output[next_idx])
    input_sentence = output[next_idx]


First sentence :  옛날하고도 아주 먼 옛날, 호랑이가 담배 피우던 때였어요.
Generated 1 : 토끼가 금연을 하고 나자 호랑이는 금방 눈을 떴고 토끼는 기분이 좋아졌어요.
Generated 2 : 토끼는 형제에게 함께 담배 피우자고 결심하고는 형제에게 말했어요.
Generated 3 : 한 토끼가 담배 피우던 때였어요.

Choose the sentence number : 3
Selected sentence :  한 토끼가 담배 피우던 때였어요.
Generated 1 : 한 토끼가 호랑이 담배 피우던 때를 생각하며 호랑이가 말했어요.
Generated 2 : 한 토끼가 호랑이가 나타나 눈을 동그랗게 뜨고 바라보자, 그 앞에 웬 호랑이가 떡 하니 앉아 있는 거예요.
Generated 3 : 한 토끼가 호랑이 담배 피우던 때였어요.


KeyboardInterrupt: ignored